# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [4]:
import os

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
from torchvision import datasets, transforms, models

import numpy as np

import matplotlib.pyplot as plt

from collections import OrderedDict

import json

import time

from PIL import Image

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [5]:
# defines data directories
data_dir = 'flowers'
print(data_dir)
train_dir = data_dir + '/train'
print(train_dir)
valid_dir = data_dir + '/valid'
print(valid_dir)
test_dir = data_dir + '/test'
print(test_dir)

flowers
flowers/train
flowers/valid
flowers/test


In [6]:
# defines transforms for the training, validation, and testing sets
training_transforms = transforms.Compose([transforms.RandomRotation(30),
                                          transforms.RandomResizedCrop(224),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])
valtest_transforms = transforms.Compose([transforms.Resize(256),
                                         transforms.CenterCrop(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.485, 0.456, 0.406],
                                                              [0.229, 0.224, 0.225])])

# loads the datasets with ImageFolder
training_data = datasets.ImageFolder(train_dir, transform = training_transforms)
print(training_data)
validation_data = datasets.ImageFolder(valid_dir, transform = valtest_transforms)
print(validation_data)
testing_data = datasets.ImageFolder(test_dir, transform = valtest_transforms)
print(testing_data)

# uses the image datasets and the tranforms to define the dataloaders
trainloader = torch.utils.data.DataLoader(training_data, batch_size = 64, shuffle = True)
validloader = torch.utils.data.DataLoader(validation_data, batch_size = 64, shuffle = True)
testloader = torch.utils.data.DataLoader(testing_data, batch_size = 64, shuffle = True)

Dataset ImageFolder
    Number of datapoints: 6552
    Root Location: flowers/train
    Transforms (if any): Compose(
                             RandomRotation(degrees=(-30, 30), resample=False, expand=False)
                             RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                             RandomHorizontalFlip(p=0.5)
                             ToTensor()
                             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                         )
    Target Transforms (if any): None
Dataset ImageFolder
    Number of datapoints: 818
    Root Location: flowers/valid
    Transforms (if any): Compose(
                             Resize(size=256, interpolation=PIL.Image.BILINEAR)
                             CenterCrop(size=(224, 224))
                             ToTensor()
                             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
       

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [7]:
# loads category-to-name .json file
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# prints category names
print(cat_to_name)
# prints quantities associated to each category name
print("\n Length:", len(cat_to_name))

{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54': 's

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

<font color='red'>**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.</font>

In [15]:
# initialises variables to be used in Part II as argparse variables
# uses VGG-19 because VGG-11 performed POORLY
arch = 'vgg19'
lr = .001
hidden_layer = 1024
gpu = 'gpu'
epochs = 6
dropout = 0.5

# builds the classifier
# defines classifier function
def Classifier(arch='vgg19', dropout = 0.5, hidden_layer = 1024):
    # possible choices of pretrained architectures
    if arch == 'vgg19':
        model = models.vgg19(pretrained = True)
        input_size = 25088
    elif arch == 'densenet121':
        model = models.densenet121(pretrained = True)
        input_size = 1024
    elif arch == 'alexnet':
        model = models.alexnet(pretrained = True)
        input_size = 9216
    
    # freezes parameters to avoid backpropping through them
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    # defines a new untrained feed-forward network as a classifier
    # utilises ReLU activations and dropouts
    my_classifier = nn.Sequential(OrderedDict([
                            ('fc1', nn.Linear(25088, 1024)),
                            ('drop', nn.Dropout(p=0.5)),
                            ('relu', nn.ReLU()),
                            ('fc2', nn.Linear(1024, 102)),
                            ('output', nn.LogSoftmax(dim=1))
                            ]))

    model.classifier = my_classifier

    return model

model = Classifier()

print('\nThe classifier has been correctly built.')
print(f'\nThe model\'s characteristics are as follows:\n{model}')


The classifier has been correctly built.

The model's characteristics are as follows:
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv

In [ ]:
# establishes criterion and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr)

# creates training
# defines the function to train the model
def train_model(model = model, trainloader = trainloader, criterion = criterion, optimizer = optimizer, epochs = epochs, gpu = gpu):
    steps = 0
    print_every = 40
    
    # sets to gpu per user input
    if gpu == 'gpu':
        model.to('cuda')
        
    # defines starting time
    start_training_time = time.time()
    # prints message to inform that training is in progress
    print('The training has now started.\n')
        
    # begins epoch passes
    for e in range(epochs):
        running_loss = 0
        
        # prints epoch metrics
        print('Epoch: {} / {}.\n'.format(e + 1, epochs))
        
        for images, labels in trainloader:
            # increases step number
            steps += 1
            
            # moves images and labels to GPU per user input
            if gpu == 'gpu':
                images, labels = images.to('cuda'), labels.to('cuda')
                
            # zeros out the optimizer's gradient
            optimizer.zero_grad()
            
            # calculates loss and updating weights
            outputs = model.forward(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # adjusts running loss
            running_loss += loss.item()
            
            # calculates validation loss and accuracy every 20 steps
            if steps % print_every == 0:
                # moves model to .eval mode
                model.eval()
                
                val_loss = 0
                accuracy = 0
                
                # calculates validation loss and accuracy
                for val_images, val_labels in validloader:
                    
                    # moves to GPU per user input
                    if gpu == 'gpu':
                        val_images, val_labels = val_images.to('cuda'), val_labels.to('cuda')
                        
                    # turns off gradient for validation purposes
                    with torch.no_grad():
                        val_outputs = model.forward(val_images)
                        val_loss = criterion(val_outputs, val_labels)
                        
                        # calculates probability from the val_outputs
                        ps = torch.exp(val_outputs)
                        top_p, top_class = ps.topk(1, dim = 1)
                        equals = top_class == val_labels.view(*top_class.shape)
                        accuracy += torch.mean(equals.type(torch.FloatTensor))
                
                # metrics
                # prints out the training loss metrics
                print('Training Loss: {:.3f}.'.format(running_loss))
                # prints out the validation loss metrics
                print('Validation Loss: {:.3f}.'.format(val_loss))
                # prints out the validation accuracy metrics
                print('Validation Accuracy: {:.3f}.\n'.format(accuracy))
                
                # resets running loss to zero
                running_loss = 0
                # turns the dropouts back on
                model.train()

# trains the model
train_model(model, trainloader, criterion, optimizer, epochs, gpu)

# sets ending time
end_training_time = time.time()
print('\nThe training has ended.')

# calculates and prints running time
training_time = end_training_time - start_training_time
print('Total training time: {:.0f}m {:.0f}s'.format(training_time / 60, training_time % 60))              

The training has now started.

Epoch: 1 / 6.

Training Loss: 52.265.
Validation Loss: 0.234.
Validation Accuracy: 11.177.

Training Loss: 50.076.
Validation Loss: 0.548.
Validation Accuracy: 11.195.

Epoch: 2 / 6.

Training Loss: 21.978.
Validation Loss: 0.592.
Validation Accuracy: 11.148.

Training Loss: 49.534.
Validation Loss: 0.767.
Validation Accuracy: 11.313.

Training Loss: 49.690.
Validation Loss: 0.746.
Validation Accuracy: 11.025.

Epoch: 3 / 6.

Training Loss: 44.241.
Validation Loss: 0.474.
Validation Accuracy: 11.469.



## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [19]:
# validation test method
# performs validation on the test set

def test_checker(testloader = testloader, gpu = gpu):
    correct = 0
    total = 0
    
    # moves to GPU based on user input
    if gpu == 'gpu':
        model.to('cuda')
     
    # sets start time
    start_time = time.time()
    # informs that the validation has started
    print("The validation has started...")
    
    # tests with the gradients turned off
    with torch.no_grad():
        for images, labels in testloader:
            if gpu == 'gpu':
                images, labels = images.to('cuda'), labels.to('cuda')
                
            # calculates model accuracy
            # satisfaction level is 70%
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
           
        # sets end time and informs validation has ended
        end_time = time.time()
        print('The validation has ended.')
        # calculates and prints running time for validation task
        validation_time = end_time - start_time
        print('\nTime elapsed for validation: {:.0f}m {:.0f}s'.format(validation_time / 60, validation_time % 60))
        
        # prints out network accuracy
        print('Accuracy of the network on test data set: {}%'.format((correct / total) * 100))

The validation has started...
The validation has ended.

Time elapsed for validation: 0m 21s

Accuracy Test value: 0.9243


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [3]:
# creation and saving of a checkpoint

# attaches mapping of classes to indices from image dataset
model.class_to_idx = training_data.class_to_idx

# defines checkpoint
checkpoint = {'arch': arch,
              'lr': lr,
              'hidden_layer': hidden_layer,
              'gpu': gpu,
              'epochs': epochs,
              'dropout': dropout,
              'state_dict': model.state_dict(),
              'class_to_idx': model.class_to_idx}

#saves checkpoint
torch.save(checkpoint, 'checkpoint.pth')

NameError: name 'image_datasets' is not defined

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [4]:
# rebuilds the model from saved checkpoint

# defines model to be rebuilt
def load_model(filepath):
    checkpoint = torch.load(filepath)
    
    arch = checkpoint['arch']
    lr = checkpoint['lr']
    hidden_layer = checkpoint['hidden_layer']
    gpu = checkpoint['gpu']
    epochs = checkpoint['epochs']
    dropout = checkpoint['dropout']
    state_dict = checkpoint['state_dict']
    class_to_idx = checkpoint['class_to_idx']
    
    # defines model and model.class_to_idx
    model = Classifier(arch, dropout, hidden_layer)
    model.class_to_idx = class_to_idx
    # loads state_dict as defined above
    model.load_state_dict(state_dict)
    
    # returns the model
    return model

# defines loaded_model and prints it
loaded_model = load_model('checkpoint.pth')
print(loaded_model)

NameError: name 'torch' is not defined

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # processes a PIL image for use in a PyTorch model
    # defines picture
    picture = Image.open(image)
    
    # normalises PIL image for PyTorch
    transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224), 
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],
                                                         [0.229, 0.224, 0.225])])
    
    # defines numpy array
    np_array = transform(picture).float()
    
    # returns numpy array
    return np_array

# defines processed image
processed_image = (test_dir + '/1/' + 'image_06764.jpg')
processed_image = process_image(processed_image)

# prints out image for verification
print(processed_image.shape)

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
# defines imshow function for tensorflow
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch assumes the colour channel is the first dimension
    # matplotlib assumes the colour channel is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # undoes preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # the image needs to be clipped between 0 and 1 
    # otherwise it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    # returns image
    return ax

In [ ]:
# testes the imshow function
imshow(process_image(test_dir + '/43/' + 'image_02365.jpg'));

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    
    #Processing image
    image = process_image(image_path)
    image = image.float().unsqueeze_(0)
    
    #Moving to GPU if user prompted
    if gpu == 'gpu':
        model.to('cuda:0')
        
    #Creating prediction score
    with torch.no_grad():
        if gpu == 'gpu':
            image = image.to('cuda')
            
        output = model.forward(image)
        
    prediction = F.softmax(output.data, dim = 1)
    
    probs, indices = prediction.topk(topk)
    probs = probs.cpu().numpy()[0]
    indices = indices.cpu().numpy()[0]
    
    idx_to_class = {v:k for k, v in model.class_to_idx.items()}
    classes = [idx_to_class[x] for x in indices]
    
    return probs, classes

#Testing out the 'predict' function
test_image = (test_dir + '/43/' + 'image_02431.jpg')
probs, classes = predict(test_image, loaded_model)
print(probs)
print(classes)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
def check_sanity(test_image = test_image, model = loaded_model):
    probs, classes = predict(test_image, model)
    flower_names = [cat_to_name[str(i)] for i in classes]
    
    #Creating our grids with matplotlib
    fig = plt.figure(figsize = (8, 8))
    ax1 = plt.subplot2grid((15, 9), (0,0), colspan = 9, rowspan = 9)
    ax2 = plt.subplot2grid((15, 9), (10,2), colspan = 5, rowspan = 5)
    
    #Showing the image itself
    image = Image.open(test_image)
    ax1.axis('off')
    ax1.set_title('Image of the Test Flower')
    ax1.imshow(image)
    
    #Showing the prediction graph
    y_ticks = np.arange(5)
    ax2.set_yticks(y_ticks)
    ax2.set_yticklabels(flower_names)
    ax2.set_title('Probability Chart')
    ax2.set_xlabel('Probability')
    ax2.set_ylabel('Flower Types')
    ax2.barh(y_ticks, probs, align = 'center')
    
    plt.show
    
check_sanity()

<font color='red'>**Reminder for Workspace users:** If your network becomes very large when saved as a checkpoint, there might be issues with saving backups in your workspace. You should reduce the size of your hidden layers and train again. 
    
We strongly encourage you to delete these large interim files and directories before navigating to another page or closing the browser tab.</font>